## Compare geometries and store object IDs

This notebook uses cursors and geometry objects to acquire the geometry of a HUC08 polygon and update the associated polygon in a local feature class to match the master copy.
The master datset is a feature layer on ArcGIS Online.

In [49]:
# This data is in WGS 1984 Web Mercator Aux. Sphere
url = 'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/Watershed_Boundary_Dataset_HUC_8s/FeatureServer/0'
# This data is in NAD 1983 StatePlane California VI FIPS 0406 (US Feet)
messyWatershed = r'C:\DemoData\PYTS\SanDiego.gdb\Watersheds_sample'

To save on processing time, select the MUC08 features that intersect the feature class being updated. This saves time since the script later iterates over every row of a cursor, looking to see if the geometry passes a test. We are working with HUC08 features in San Diego county area, not the entire USA.

In [50]:
masterHUC08 = arcpy.management.SelectLayerByLocation(in_layer=url, overlap_type="INTERSECT", select_features=messyWatershed)

In [51]:
# Create a spatial reference object for NAD 1983 StatePlane California VI FIPS 0406 (US Feet)
sp = arcpy.SpatialReference(2230)

# Create cursors for both datasets. Object ID is for reference.
uCur = arcpy.da.UpdateCursor(messyWatershed, ['Shape@','OBJECTID'])
sCur = arcpy.da.SearchCursor(masterHUC08, ['Shape@','OBJECTID'])

# Iterate over every feature in the local feature class
for uRow in uCur:
    print(f"Object ID of watershed to update: {uRow[1]}")
    # For each feature, iterate over the features resulting from the select by location tool. 
    for sRow in sCur:
        # We need a way to align the HUC08 features from both datasets. Geometry does not align, hence this script.
        # Ideally, there would be a unique identifying field that is reliable. We don't have that.
        # Instead, we will determine which features between the datasets are "the same" HUC08, despite the local source having messy geometry
        # Check to see if the centroid of the local feature is within the HUC08 polygon from the master source.
        # And, project the master data into the spatial reference of the local data
        if sRow[0].projectAs(sp).contains(uRow[0].centroid):
            print(f"The watershed is in {sRow[1]}")
            uRow[0] = sRow[0].projectAs(sp)
            uCur.updateRow(uRow)
            print("Update made")
        else:
            pass
    sCur.reset()
del uCur, sCur

Object ID of watershed to update: 1
The watershed is in 1330
Update made
Object ID of watershed to update: 2
The watershed is in 1332
Update made
Object ID of watershed to update: 4
The watershed is in 1878
Update made
Object ID of watershed to update: 6
The watershed is in 1880
Update made
Object ID of watershed to update: 7
The watershed is in 1881
Update made
Object ID of watershed to update: 8
The watershed is in 4
Update made
Object ID of watershed to update: 9
The watershed is in 4
Update made
Object ID of watershed to update: 10
The watershed is in 179
Update made
Object ID of watershed to update: 11
The watershed is in 9
Update made
Object ID of watershed to update: 12
The watershed is in 4
Update made
Object ID of watershed to update: 13
The watershed is in 10
Update made
Object ID of watershed to update: 14
The watershed is in 9
Update made
Object ID of watershed to update: 15
The watershed is in 10
Update made
Object ID of watershed to update: 16
The watershed is in 10
Updat